In [1]:
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from sklearn.cluster import DBSCAN
from imutils.video import FPS
from imutils import paths
import dlib.cuda as cuda
import face_recognition
import numpy as np
import argparse
import imutils
import time
import cv2
import os

In [2]:
class FaceClusteringAgent:
    """DBSCAN based Blustering"""
    
    def __init__(self):
        self.known = dict()
        self.unknown = dict()
        self.noise = dict()
        
    def predict(self, new_encoding):
        label = -1
        encoding_dict = self.unknown
        
        if not encoding_dict:
            encoding_dict[len(encoding_dict)] = [new_encoding]

        else:
            for id in encoding_dict:
                known_encodings = encoding_dict[id] 
                results = face_recognition.compare_faces(known_encodings, new_encoding)
                if all(results):
                    encoding_dict[id] += [new_encoding]
                    label = id
            if(label == -1): 
                encoding_dict[len(encoding_dict)] = [new_encoding]            
            
        return label
        
    def fit(self, new_encoding):
        return self.predict(new_encoding)

In [3]:
def main(fca):
    
    print("[INFO] starting video stream...")
    vs = VideoStream(src=0).start()
    time.sleep(2.0)
    fps = FPS().start()


    while True:
        # getFrame
        frame = vs.read()
        frame = imutils.resize(frame, width=500)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes = face_recognition.face_locations(rgb,model="cnn")

        # compute the facial embedding for the face
        ids = []
        new_encodings = face_recognition.face_encodings(rgb, boxes)
        if new_encodings:
            for new_encoding in new_encodings:
                ids.append(fca.fit(new_encoding))

        for i, box in enumerate(boxes):
                # coordinates
                (top, right, bottom, left) = box
                y = top - 10 if top - 10 > 10 else top + 10

                # perform classification to recognize the face
                name = "ID"
                text = "{}: {}".format(name, ids[i])

                # plot at image
                cv2.rectangle(frame, (right,top), (left, bottom), (0, 0, 255), 2)
                cv2.putText(frame, text, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 200), 2)

#         cv2.putText(frame, 'Counting: {}'.format(len(labelSet)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 100, 200), 2)

        
        # update the FPS counter
        fps.update()

        # show the output frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    # stop the timer and display FPS information
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    # do a bit of cleanup
    cv2.destroyAllWindows()
    vs.stop()

In [4]:
fca = FaceClusteringAgent()
main(fca)

[INFO] starting video stream...
[INFO] elasped time: 39.41
[INFO] approx. FPS: 12.74
